In [17]:
import sklearn as sk
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
import statsmodels.formula.api as smf

In [43]:
boston_dataset = load_boston()
columns = boston_dataset.feature_names
data = pd.DataFrame(boston_dataset.data, columns=columns)
median_home_values = pd.DataFrame(boston_dataset.target, columns=['median_home_values'])
data = data.join(median_home_values)

In [44]:
def forward_selected(data, response):
    """Linear model designed by forward selection.

    Parameters:
    -----------
    data : pandas DataFrame with all possible predictors and response

    response: string, name of response column in data

    Returns:
    --------
    model: an "optimal" fitted statsmodels linear model
           with an intercept
           selected by forward selection
           evaluated by adjusted R-squared
    """
    remaining = set(data.columns)
    remaining.remove(response)
    selected = []
    current_score, best_new_score = 0.0, 0.0
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            formula = "{} ~ {} + 1".format(response,
                                           ' + '.join(selected + [candidate]))
            score = smf.ols(formula, data).fit().rsquared_adj
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates.pop()
        if current_score < best_new_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
    formula = "{} ~ {} + 1".format(response,
                                   ' + '.join(selected))
    model = smf.ols(formula, data).fit()
    return model

In [50]:
model = forward_selected(data, 'median_home_values')

In [52]:
print(model.model.formula)
print(model.rsquared_adj)

median_home_values ~ LSTAT + RM + PTRATIO + DIS + NOX + CHAS + B + ZN + CRIM + RAD + TAX + 1
0.7348057723274565
